In [32]:
from text_preprocessing import spacy_preprocessing
import spacy
nlp = spacy.blank("fr")

In [33]:
def find_swift(text):
    text = spacy_preprocessing(text)
    # we use spacy s entity ruler forth matching
    ruler = nlp.add_pipe("entity_ruler")
    patterns =[{"label":"Code Swift","pattern":[{"LOWER":"code","OP":"?"},{"LOWER":"swift"},{'IS_SPACE':True,'OP':'?'},{"IS_PUNCT":True,"OP":"?"},{"TEXT": {"REGEX": "[A-Za-z0-9]+"}}]}]
    ruler.add_patterns(patterns)
    tokens = nlp(text)
    return [ent[-1] for ent in tokens.ents]


In [34]:
file = 'C:/Users/PC2/Downloads/test-ex.txt'
f=open(file,'r')
data = f.read()
find_swift(data)


[agilamcxx, creschzz80a]